In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from conformal_sparsemax.classifier import CNN,get_data,train,evaluate

In [3]:
import torch
from torch import nn
from sklearn.metrics import f1_score

In [4]:
from entmax.losses import SparsemaxLoss, Entmax15Loss

In [13]:
loss = 'softmax'
dataset = 'CIFAR100'

if loss == 'sparsemax':
    criterion = SparsemaxLoss()
elif loss == 'entmax15':
    criterion = Entmax15Loss()
elif loss == 'softmax':
    criterion = torch.nn.NLLLoss()
train_dataloader, dev_dataloader, test_dataloader, _ = get_data(0.2,16,dataset = dataset)

if dataset == 'CIFAR100':
    model = CNN(loss)
elif dataset == 'MNIST':
    model = CNN(loss,n_classes=10,input_size=28,channels=1)
else:
    raise Exception('Wrong dataset name')
    
model, train_history, val_history, f1_history = train(model,
                                            train_dataloader,
                                            dev_dataloader,
                                            criterion,
                                            epochs=50,
                                            patience=3)

_, predicted_labels, true_labels, test_loss = evaluate(
                                                    model,
                                                    test_dataloader,
                                                    criterion)

f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f'Test loss: {test_loss:.3f}')
print(f'Test f1: {f1:.3f}')

Files already downloaded and verified
Files already downloaded and verified
-- Epoch 1 --


2450it [00:20, 121.67it/s]


train_loss: 4.271
val_loss: 3.913
val_f1: 0.073
-- Epoch 2 --


2450it [00:25, 96.08it/s] 


train_loss: 3.701
val_loss: 3.551
val_f1: 0.134
-- Epoch 3 --


...

train_loss: 2.306
val_loss: 2.957
val_f1: 0.285
-- Epoch 10 --


2450it [00:27, 88.00it/s]


train_loss: 2.156
val_loss: 2.994
val_f1: 0.291
-- Epoch 11 --


2450it [00:26, 92.95it/s] 


train_loss: 1.991
val_loss: 3.117
val_f1: 0.283
-- Finished Training --
Test loss: 3.110
Test f1: 0.282


In [14]:
results = {
    'train_history':train_history,
    'val_history':val_history,
    'f1_history':f1_history,
}

In [15]:
import json
with open(f'results/{dataset}_{loss}_results.json', 'w') as f:
    json.dump(results, f)

In [16]:
torch.save(model.state_dict(), f'models/{dataset}_{loss}.pth')